### Sample program for Association Analysis (Market Basket Analysis) using FP-Growth  

#### Import libraries  

In [34]:
import pandas as pd
import pyfpgrowth  # https://fp-growth.readthedocs.io/en/latest/

#### Parameters  

In [35]:
csv_in = 'online_retail_small_cleaned.csv'

#### Read CSV file  

In [36]:
df = pd.read_csv(csv_in, delimiter=',', skiprows=0, header=0)
print(df.shape)
print(df.info())
display(df.head())

(59145, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59145 entries, 0 to 59144
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   InvoiceNo    59145 non-null  int64  
 1   StockCode    59145 non-null  object 
 2   Description  59145 non-null  object 
 3   Quantity     59145 non-null  int64  
 4   InvoiceDate  59145 non-null  object 
 5   UnitPrice    59145 non-null  float64
 6   CustomerID   59145 non-null  int64  
 7   Country      59145 non-null  object 
dtypes: float64(1), int64(3), object(4)
memory usage: 3.6+ MB
None


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536386,84880,WHITE WIRE EGG HOLDER,36,2010-12-01 09:57:00,4.95,16029,United Kingdom
1,536386,85099C,JUMBO BAG BAROQUE BLACK WHITE,100,2010-12-01 09:57:00,1.65,16029,United Kingdom
2,536386,85099B,JUMBO BAG RED RETROSPOT,100,2010-12-01 09:57:00,1.65,16029,United Kingdom
3,536404,22297,HEART IVORY TRELLIS SMALL,24,2010-12-01 11:29:00,1.25,16218,United Kingdom
4,536404,22771,CLEAR DRAWER KNOB ACRYLIC EDWARDIAN,12,2010-12-01 11:29:00,1.25,16218,United Kingdom


#### Check appearance of each StockCode  

In [37]:
top_sc = df['StockCode'].value_counts()
print(top_sc.size)
print(top_sc.head())

3140
85123A    315
85099B    246
22423     229
84879     213
47566     208
Name: StockCode, dtype: int64


#### Format transaction data  
Before:  
```
invoice1, stockcode1, ...  
invoice1, stockcode2, ...  
invoice2, stockcode1, ...  
invoice3, stockcode1, ...  
invoice3, stockcode3, ...  
```

After:  
```
[  
  [stockcode1, stockcode2],    
  [stockcode1],  
  [stockcode1, stockcode3],  
  ...  
]  
```

and stockcode should be interger.  

#### Assign an integer (ID) to each stockcode  

In [38]:
id2sc = sorted(list(set(df['StockCode'])))
sc2id = {}
for i in range(len(id2sc)):
    sc2id[id2sc[i]] = i

In [39]:
df['StockCode_ID'] = df['StockCode'].map(lambda x: sc2id[x])
display(df.head())

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,StockCode_ID
0,536386,84880,WHITE WIRE EGG HOLDER,36,2010-12-01 09:57:00,4.95,16029,United Kingdom,2775
1,536386,85099C,JUMBO BAG BAROQUE BLACK WHITE,100,2010-12-01 09:57:00,1.65,16029,United Kingdom,2907
2,536386,85099B,JUMBO BAG RED RETROSPOT,100,2010-12-01 09:57:00,1.65,16029,United Kingdom,2906
3,536404,22297,HEART IVORY TRELLIS SMALL,24,2010-12-01 11:29:00,1.25,16218,United Kingdom,1078
4,536404,22771,CLEAR DRAWER KNOB ACRYLIC EDWARDIAN,12,2010-12-01 11:29:00,1.25,16218,United Kingdom,1517


#### Grouping transaction data by InvoiceNo  

In [40]:
invoices = []
for r in df.groupby('InvoiceNo'):
    #print(len(r))  # debug
    #print(r[0])  # debug
    #print(r[1])  # debug
    #break  # debug
    s1 = set(r[1]['StockCode_ID'])
    invoices.append(list(s1))
print(len(invoices))
print(invoices[:10])

2757
[[2906, 2907, 2775], [1155, 2827, 1548, 140, 139, 277, 278, 279, 1695, 1183, 2610, 2611, 819, 1077, 1078, 316, 317, 2879, 2880, 2881, 2883, 1232, 992, 1517, 1518, 1519, 369, 370], [2816, 2435, 1290, 652, 1294, 2575, 2831, 529, 2835, 788, 789, 1556, 2836, 1555, 922, 1307, 1695, 1184, 674, 2604, 1455, 560, 2609, 688, 689, 55, 1463, 1465, 579, 1228, 1484, 974, 1363, 2774, 1371, 1372, 735, 1250, 1253, 1513, 238, 367, 1646, 369, 1011, 2804, 119, 1144], [1670, 1514, 1515, 789, 533, 2556], [1217, 1572, 486, 2537, 1578, 1515, 1389, 563, 1396, 1397, 1398, 1267, 1270, 1401, 1339, 1692, 1215], [1376, 1377, 322, 1378, 921, 970, 1177, 2906, 798], [640, 1059, 1527, 645, 1189, 1541, 1543, 1544, 202, 979, 1111], [1541, 1543, 1641, 2986, 1707, 1673, 723, 509, 1695], [2752, 1222, 1577, 921, 698], [704, 2722, 2724, 1445, 2501, 2876, 2858, 874, 2608, 2290, 2292, 2325, 2326, 1684, 441, 1210, 2907, 2874]]


#### Market Basket Analysis by FP-Growth  

In [41]:
%time patterns = pyfpgrowth.find_frequent_patterns(invoices, 40)

CPU times: user 346 ms, sys: 7.66 ms, total: 354 ms
Wall time: 353 ms


In [42]:
#print(patterns)

In [43]:
%time rules = pyfpgrowth.generate_association_rules(patterns, 0.8)

CPU times: user 878 µs, sys: 0 ns, total: 878 µs
Wall time: 888 µs


In [44]:
print(rules)

{(1473,): ((1475,), 0.803921568627451), (1332,): ((1331,), 0.8243243243243243), (1474, 1478): ((1475,), 0.8035714285714286), (1475, 1478): ((1474,), 0.8490566037735849), (138, 1156): ((137,), 0.8333333333333334), (140, 1156): ((137,), 0.8448275862068966), (1154, 1156): ((137,), 0.828125)}


In [45]:
results = []
for x in rules:
    ret = [x, rules[x][0], rules[x][1]]
    results.append(ret)
df_res = pd.DataFrame(results)
df_res.columns = ['LHS', 'RHS', 'Conf']

In [46]:
display(df_res.sort_values(by='Conf', ascending=False))

,LHS,RHS,Conf
3,"(1475, 1478)","(1474,)",0.849057
5,"(140, 1156)","(137,)",0.844828
4,"(138, 1156)","(137,)",0.833333
6,"(1154, 1156)","(137,)",0.828125
1,"(1332,)","(1331,)",0.824324
0,"(1473,)","(1475,)",0.803922
2,"(1474, 1478)","(1475,)",0.803571


#### Get original StockCode  

In [47]:
print(id2sc[1474])
print(id2sc[1475])
print(id2sc[1478])

22726
22727
22730


#### Calculation of Lift  

In [48]:
n_all = len(invoices)
lift = []
for i in range(df_res.shape[0]):
    rhs = df_res.at[i, 'RHS']
    conf = df_res.at[i, 'Conf']
    n_rhs = 0
    for items in invoices:
        if set(items) >= set(rhs):
            n_rhs += 1
    lift1 = conf / (n_rhs / n_all)
    lift.append(lift1)
    
df_res['Lift'] = lift

In [49]:
display(df_res.sort_values(by='Conf', ascending=False))

,LHS,RHS,Conf,Lift
3,"(1475, 1478)","(1474,)",0.849057,18.287883
5,"(140, 1156)","(137,)",0.844828,11.883621
4,"(138, 1156)","(137,)",0.833333,11.721939
6,"(1154, 1156)","(137,)",0.828125,11.648677
1,"(1332,)","(1331,)",0.824324,28.057558
0,"(1473,)","(1475,)",0.803922,16.540386
2,"(1474, 1478)","(1475,)",0.803571,16.533182
